In [ ]:
import json
import os

def merge_and_sort_jsonl_files(directory_path, output_file):
    all_data = []

    # 遍历给定路径下的所有文件
    for filename in os.listdir(directory_path):
        if filename.endswith('.jsonl'):
            # 为每个jsonl文件读取和解析每一行
            with open(os.path.join(directory_path, filename), 'r') as file:
                for line in file:
                    all_data.append(json.loads(line))
    
    # 根据'pair_id'关键字对所有数据进行排序
    sorted_data = sorted(all_data, key=lambda x: x['pair_id'])
    print(len(all_data))
    # 将排序后的数据写入新的json文件
    with open(output_file, 'w') as file:
        json.dump(sorted_data, file, ensure_ascii=False, indent=4)

# 用法示例
merge_and_sort_jsonl_files('/media/sata2/cdp/backup/MLLM-Judge-backup/Dataset/Benchmark/arxiv_data/pair', 'pair.json')


In [ ]:
import json

def convert_json_to_jsonl(input_json_path, output_jsonl_path):
    # 读取JSON文件
    with open(input_json_path, 'r', encoding='utf-8') as json_file:
        data = json.load(json_file)  # 假设这是一个JSON对象数组

    # 确保读取到的数据是一个列表
    if not isinstance(data, list):
        raise ValueError("Expected a JSON array in the input file.")

    # 将每个JSON对象写入JSONL文件的单独一行
    with open(output_jsonl_path, 'w', encoding='utf-8') as jsonl_file:
        for item in data:
            jsonl_file.write(json.dumps(item, ensure_ascii=False) + '\n')

# 使用示例
convert_json_to_jsonl('/media/ssd/cdp/MLLM-Judge/Dataset/Benchmark/pair.json', 'pair.jsonl')


In [ ]:
with open("/media/ssd/cdp/MLLM-Judge/Dataset/step3/Score/Gemini.jsonl", 'r') as f:
    data = [json.loads(line) for line in f]


In [ ]:
for i in data:
    if int(i['result']['judge']) > 5:
        print(i['result'])
        i['result']['judge'] = str(i['result']['judge'][-1])

In [ ]:
# 将修改后的数据写入新的json文件
with open("/media/ssd/cdp/MLLM-Judge/Dataset/step3/Score/Gemini.jsonl", 'w') as file:
    for i in data:
        json.dump(i, file, ensure_ascii=False)
        file.write('\n')

In [113]:
with open('/media/ssd/cdp/MLLM-Judge/Dataset/Benchmark/pair.jsonl', 'r') as f:
    gt = [json.loads(line) for line in f]

with open('/media/ssd/cdp/MLLM-Judge/Dataset/step3/Pair/Gemini.jsonl', 'r') as f:
    data = [json.loads(line) for line in f]
    
print(len(gt))
len(data)

5696


5745

In [ ]:
for i in data:
    if 'result' not in list(i.keys()):
        print(i)

In [114]:
cnt = 0
reversed = 0
for i in gt:
    for j in data:
        if i['pair_id'] == j['pair_id']:
            if i['answer1']['name'] != j['answer1']['name'] or i['answer2']['name'] != j['answer2']['name']:
                cnt+=1
            if i['answer1']['name'] == i['answer2']['name'] or j['answer2']['name'] == j['answer1']['name']:
                reversed+=1
print(cnt)
print(reversed)

4511
10


In [115]:
revised_data = []
for j in data:
    found_match = False  # 标记是否找到匹配
    for i in gt:
        if i['id'] == j['id']:
            if i['answer1']['name'] == j['answer1']['name'] and i['answer2']['name'] == j['answer2']['name']:
                j['pair_id'] = i['pair_id']
                revised_data.append(j)
                found_match = True  # 找到匹配，将退出内层循环
            elif i['answer1']['name'] == j['answer2']['name'] and i['answer2']['name'] == j['answer1']['name']:
                j['pair_id'] = i['pair_id']
                j['answer1'], j['answer2'] = j['answer2'], j['answer1']  # 交换answers
                # 根据匹配调整result字段
                j['result']['judge'] = 'A' if j['result']['judge'] == 'B' else 'B'   
                revised_data.append(j)
                found_match = True  # 找到匹配，将退出内层循环
        if found_match:
            break  # 退出内层循环
                
len(revised_data)

KeyError: 'result'

In [112]:
# rerank the revised_data by pair_id
revised_data = sorted(revised_data, key=lambda x: x['pair_id'])

# write revised_data to a new jsonl file
with open("/media/ssd/cdp/MLLM-Judge/Dataset/step3/Pair/new CogVLM.jsonl", 'w') as file:
    for i in revised_data:
        json.dump(i, file, ensure_ascii=False)
        file.write('\n')

In [98]:
import json

def check_duplicate_pair_ids(jsonl_path):
    pair_id_counts = {}
    duplicates = []

    # 读取JSONL文件并统计pair_id出现的次数
    with open(jsonl_path, 'r', encoding='utf-8') as file:
        for line in file:
            data = json.loads(line)
            pair_id = data.get('pair_id')
            if pair_id in pair_id_counts:
                pair_id_counts[pair_id] += 1
            else:
                pair_id_counts[pair_id] = 1

    # 重新遍历文件，查找具有相同pair_id的元素
    with open(jsonl_path, 'r', encoding='utf-8') as file:
        for line in file:
            data = json.loads(line)
            pair_id = data.get('pair_id')
            # 如果pair_id的计数超过1，则记录下来
            if pair_id_counts[pair_id] > 1 and pair_id not in duplicates:
                print(f"Duplicate pair_id found: {pair_id} with data: {data}")
                duplicates.append(pair_id)
    print(len(pair_id_counts.items()))

# 使用示例
check_duplicate_pair_ids('/media/ssd/cdp/MLLM-Judge/Dataset/step3/Pair/GPT-4V(ision).jsonl')


6151


In [76]:
import json

def remove_duplicate_pair_ids(jsonl_path):
    seen_pair_ids = set()
    unique_entries = []

    # 读取并处理JSONL文件，只保留第一个出现的具有特定pair_id的条目
    with open(jsonl_path, 'r', encoding='utf-8') as file:
        for line in file:
            data = json.loads(line)
            pair_id = data.get('pair_id')
            if pair_id not in seen_pair_ids:
                seen_pair_ids.add(pair_id)
                unique_entries.append(data)

    # 将处理后的数据写回原文件
    with open(jsonl_path, 'w', encoding='utf-8') as file:
        for entry in unique_entries:
            file.write(json.dumps(entry, ensure_ascii=False) + '\n')

# 使用示例
remove_duplicate_pair_ids('/media/ssd/cdp/MLLM-Judge/Dataset/Benchmark/pair.jsonl')
